# NXC and Execo

## Some Imports

First import the usual Execo tools to run an experiment 

In [1]:
from execo import Remote
from execo_g5k import oardel, oarsub, OarSubmission
from execo_engine import Engine, logger

We now import our `nxc` function that returns the OAR nodes

In [2]:
from nixos_compose.nxc_execo import get_oar_job_nodes_nxc

## Definition of the Execo Experiment Engine

Execo provides the notion of `Engine` for running experiments.

We have to define a subclass of this `Engine` class, and implement some of its methods.

In [3]:
class MyEngine(Engine):
    def __init__(self):
        super(MyEngine, self).__init__()
        # We add here a CLI argument to use a defined compose-info file
        # by default it is looking at the nxc/build folder
        parser = self.args_parser
        parser.add_argument('--nxc_build_file', help='Path to the NXC compose info file')
        self.nodes = {}

    def init(self):
        # Here we setup the reservation:
        # how many nodes, where, for how long, etc
        nb_nodes = 2
        site = "grenoble"
        cluster = "dahu"
        # makes the reservation
        oar_job = reserve_nodes(nb_nodes, site, cluster)
        job_id, site = oar_job[0]
        # We now define the number of nodes we want to be assign to each role.
        # Here we have a single role: "foo"
        # and we assign 2 nodes to this role
        roles_quantities = { "foo": 2 }
        # We now call our nxc function to make the mapping between the reserved nodes and the roles
        # while setting up the correct environnment per node
        self.nodes = get_oar_job_nodes_nxc(job_id, site, self.args.nxc_build_file, roles_quantities=roles_quantities)
        # self.nodes is a dict where the keys are roles and the values lists of execo.Host

    def run(self):
        # once everything is set up, we can execute some commands
        # here we execute a simple command on all the nodes having the role "foo"
        my_command = "echo \"Hello from $(whoami) at $(hostname) ($(ip -4 addr | grep \"/20\" | awk '{print $2;}'))\" > /tmp/hello"
        hello_remote = Remote(my_command, self.nodes["foo"], connection_params={'user': 'root'})
        hello_remote.run()
        
        my_command2 = "cat /tmp/hello"
        cat_remote = Remote(my_command2, self.nodes["foo"], connection_params={'user': 'root'})
        cat_remote.run()
        for process in cat_remote.processes:
            print(process.stdout)
        
def reserve_nodes(nb_nodes, site, cluster, walltime=3600):
    # A small helper function to make the reservation
    jobs = oarsub([(OarSubmission("{{cluster='{}'}}/nodes={}".format(cluster, nb_nodes), walltime, job_type=["day"]), site)])
    return jobs

## Running our Engine

We will be using this compose info file:

In [4]:
nxc_build_file = "./compose-info.json"
with open(nxc_build_file, "r") as compose_info_content:
    import json
    json_content = json.load(compose_info_content)
    pp_content = json.dumps(json_content, indent=4)
    print(pp_content)

{
    "all": {
        "initrd": "/nix/store/dz3bm0287fmpq29mmbjrg9qyjzgh5lck-initrd/initrd",
        "kernel": "/nix/store/i9k6v2297smxh3x06h7j8flkc4yllbjq-image/kernel",
        "qemu_script": "/nix/store/n5yz728dkhmck8izppzaifar1dw3m2g3-qemu_script",
        "stage1": "/nix/store/1cixs6y21klz4vm54590mcfcqrv20wl0-stage-1-init.sh"
    },
    "compositions_info": {
        "composition": {
            "all_store_info": "/nix/store/7kj72l0v51j0c5ixvg832fi7s0qq5hmd-all-store-info",
            "nodes": {
                "foo": {
                    "closure_info": "/nix/store/bb412bcibzyid6pn51qmslxwzg37jb08-closure-info",
                    "init": "/nix/store/lqq5jkwcmn5gmms9cy5m1blf6gj0aacb-nixos-system-unnamed-21.11pre-git/init"
                }
            },
            "test_script": "/nix/store/q4g5ynkirk3pl8szfaa8m2pq3zcjafml-test-script"
        }
    },
    "compositions_squashfs_store": "/nix/store/7sarczwj5pcf3cpmr7cfz6817aawkn56-all-compositions-squashfs.img",
    "flavou

Once our `Engine` is ready we can run it

In [5]:
# Defines an instance of our Engine ...
ENGINE = MyEngine()
# ... and starts it
ENGINE.start(["--nxc_build_file", nxc_build_file])

2021-11-03 15:09:58,374 INFO: command line arguments: ['/home/qguilloteau/venv-jupyter/lib/python3.7/site-packages/ipykernel_launcher.py', '-f', '/tmp/tmpa1q1m923.json', '--HistoryManager.hist_file=:memory:']
2021-11-03 15:09:58,375 INFO: command line: /home/qguilloteau/venv-jupyter/lib/python3.7/site-packages/ipykernel_launcher.py -f /tmp/tmpa1q1m923.json --HistoryManager.hist_file=:memory:
2021-11-03 15:09:58,376 INFO: run in directory /home/qguilloteau/nix/nixos-compose/MyEngine_20211103_150958_+0100
compose info file: ./compose-info.json
G5K nodes: [Host('dahu-29.grenoble.grid5000.fr'), Host('dahu-3.grenoble.grid5000.fr')]
Launch ssh(s) kexec
Waiting ssh ports:
✔ All ssh ports are opened
Hello from root at foo (172.16.20.29/20)

Hello from root at foo (172.16.20.3/20)

